In [1]:
import pandas as pd
from pathlib import Path
import numpy as np


In [2]:
file_path = "../data/master_residue_file.csv"
balanced_fp = "../data/balanced_master_residue_file.csv"
chunk_size  = 100_000
label_col   = "label"
drop_cols   = ['amino_acid', 'dssp_residue_name']

In [3]:
# ── Pass 1: count positives & unlabeled ────────────────────────────────
pos_cnt = unl_cnt = 0
for c in pd.read_csv(file_path, chunksize=chunk_size, usecols=[label_col]):
    pos_cnt += (c[label_col] == 1).sum()
    unl_cnt += (c[label_col] == 0).sum()

In [4]:
sample_prob = pos_cnt / unl_cnt          # probability to keep each unlabeled row
rng = np.random.default_rng(42)          # reproducible sampling

In [5]:

# ── Pass 2: write balanced file chunk-by-chunk ─────────────────────────
Path(balanced_fp).unlink(missing_ok=True)  

In [6]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunk = chunk.drop(columns=drop_cols, errors="ignore")

    pos_rows = chunk[chunk[label_col] == 1]
    unl_rows = chunk[chunk[label_col] == 0]

    # Bernoulli sampling keeps ≈ pos_cnt unlabeled rows overall
    keep_mask = rng.random(len(unl_rows)) < sample_prob
    unl_sample = unl_rows[keep_mask]

    balanced_chunk = pd.concat([pos_rows, unl_sample], ignore_index=True)

    # append to balanced CSV without headers after the first write
    balanced_chunk.to_csv(
        balanced_fp,
        mode="a",
        index=False,
        header=not Path(balanced_fp).exists()
    )

/tmp/ipykernel_5913/3804527052.py:1: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
/tmp/ipykernel_5913/3804527052.py:1: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
/tmp/ipykernel_5913/3804527052.py:1: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
/tmp/ipykernel_5913/3804527052.py:1: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chunk_size):
/tmp/ipykernel_5913/3804527052.py:1: DtypeWarning: Columns (55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_path, chunksize=chu